In [12]:
import tkinter as tk
from tkinter import *
from PIL import Image, ImageTk
import tkinter.font as font
import mysql.connector
from datetime import date
import sqlite3 as sql
import pandas as pd
from tkinter import messagebox, ttk
import cv2
import os
from PIL import Image
import numpy as np
from datetime import datetime
from datetime import date, timedelta
import time
import random
import math
import smtplib
mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition") 
mycursor = mydb.cursor() 

from tkinter import messagebox as mess
import tkinter.simpledialog as tsd
import csv
import datetime
import time

global key
key = ''

ts = time.time()
date = datetime.datetime.fromtimestamp(ts).strftime('%d-%m-%Y')
day,month,year=date.split("-")

mont={'01':'January',
      '02':'February',
      '03':'March',
      '04':'April',
      '05':'May',
      '06':'June',
      '07':'July',
      '08':'August',
      '09':'September',
      '10':'October',
      '11':'November',
      '12':'December'
      }

def tick():
    time_string = time.strftime('%H:%M:%S')
    clock.config(text=time_string)
    clock.after(200,tick)

In [13]:
def detect_face(emp_type):
    def draw_rect(img,classifier,clf):
        gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        roi = classifier.detectMultiScale(gray_image,1.3,5)
        for(x,y,w,h) in roi:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        return gray_image
    def recognizer(gray_image,clf,faceCascade):
        roi = faceCascade.detectMultiScale(gray_image,1.3,5)
        today = datetime.date.today()
        global name
        name = "Unknown"
        for(x,y,w,h) in roi:
            id,pred = clf.predict(gray_image[y:y+h,x:x+w])
            confidence = int(100 * (1 - pred / 300))
            print("conf "+str(confidence))
            print("pred "+str(pred))
            print("id "+str(id))
            mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition")
            mycursor = mydb.cursor()
            sql = "select type from users where enrollment_no = %s"
            val = (str(id),)
            mycursor.execute(sql,val)
            db_type = mycursor.fetchone()
            if (confidence > 75 and emp_type == db_type[0]):
                mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition")
                mycursor = mydb.cursor()
                sql = "select name from attendence where enrollment_no = %s"
                val = (str(id),)
                mycursor.execute(sql,val)
                name = mycursor.fetchone()
                messagebox.showinfo('Message','Hello '+str(name[0])+' Your Attendence taken Succcessfully...')
                mycursor = mydb.cursor()
                sql = "update attendence set `%s` = 'present' where enrollment_no = %s"
                val = (str(today),str(id),)
                mycursor.execute(sql,val)
                mydb.commit()
                if(emp_type == "admin"):
                    admin()
            else:
                messagebox.showinfo('Message','Access Denied Press Enter...'+str(db_type[0]))

    faceCascade = cv2.CascadeClassifier("haar-cascade-files-master/haarcascade_frontalface_default.xml")
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")
    video_capture = cv2.VideoCapture(0)
    while True:
        ret, image = video_capture.read()
        gray_image = draw_rect(image,faceCascade,clf)
        cv2.putText(image,"Press Enter to Recognize...",(30,30),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,0),1,cv2.LINE_AA)
        cv2.imshow("Face Detection",image)
        if cv2.waitKey(1) == 13:
            temp_img = gray_image
            video_capture.release()
            cv2.destroyAllWindows()
            break
    # plt.imshow(image)
    # plt.imshow(temp_img)
    # plt.imshow(gray_image)
    recognizer(temp_img,clf,faceCascade)

In [14]:
def train_model():
    data = "faces"
    path = os.listdir(data)
    faces = []
    ids = []
    img = path[0]
#     print(path)
    for image in path:
        image = "faces/"+image
#         print(image)
        img = Image.open(image).convert('L')
        imgNp = np.array(img,'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        
        faces.append(imgNp)
        ids.append(id)
    
    ids = np.array(ids)
    
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
    messagebox.showinfo('Message','User Registered Successfully...')
    
def details(en_no,name,ty_pe,department,email,phn,address,pincode):

    server = smtplib.SMTP_SSL("smtp.gmail.com", 465)    
    user = "milanraghvani75@gmail.com"
    password = "opspfjtkvtohcjeh"

    server.login(user, password)

    server.sendmail("milanraghvani75@gmail.com",
                    email,
                    "Your Registered Details :"+"\n"+"\n"+"Id = "+en_no+"\n"+"Name = "+name+"\n"+"Type = "+ty_pe+"\n"+"Department = "+department+"\n"+"Email Id = "+email+"\n"+"Phone No. = "+phn+"\n"+"Address = "+address+"\n"+"Pincode = "+pincode)

    server.quit()    

In [15]:
# id as label
def generate_dataset():
#     if (a2.get()=="" and a1.get()=="" and a4.get()=="" and a5.get()=="" and a6.get()=="" and a7.get()=="" and a9.get()==""):
#         messagebox.showinfo('Message','Please provide complete information')
    if(a2.get()==""):
        messagebox.showinfo('Message','Please provide Employee Name')
    elif(a1.get()==""):
        messagebox.showinfo('Message','Please provide Employee Type')
    elif(a4.get()==""):
        messagebox.showinfo('Message','Please provide Employee Department')
    elif(a5.get()==""):
        messagebox.showinfo('Message','Please provide Employee Email Id')
    elif(a6.get()==""):
        messagebox.showinfo('Message','Please provide Employee Phone Number')
    elif(a7.get()==""):
        messagebox.showinfo('Message','Please provide Employee Address')
    elif(a9.get()==""):
        messagebox.showinfo('Message','Please provide Employee Pincode')
    else:
        name = a2.get()
        en_no = id_label['text']
        ty_pe = a1.get()
        department = a4.get()
        email = a5.get()
        phn = a6.get()
        address = a7.get()
        pincode = a9.get()
        
        mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition")
        mycursor = mydb.cursor()
        val = (en_no,)
        sql = "SELECT COUNT(*) FROM `attendence` WHERE `enrollment_no` = %s"
        mycursor.execute(sql,val)
        count = mycursor.fetchone()
        if count[0] == 1:
            messagebox.showinfo('Message','User already existing You can Directly Recognize your face')
        else :
            mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition")
            mycursor = mydb.cursor()
            sql = "insert into users(enrollment_no,name,type,department,email,phone_no,address,pincode) values(%s,%s,%s,%s,%s,%s,%s,%s)"
            val = en_no,name,ty_pe,department,email,phn,address,pincode
            mycursor.execute(sql,val)
            sql = "insert into attendence(enrollment_no,name) values(%s,%s)"
            val = en_no,name
            mycursor.execute(sql,val)
            mydb.commit()

            face_classifier = cv2.CascadeClassifier('haar-cascade-files-master/haarcascade_frontalface_default.xml')
            def face_cropped(img):
                gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                faces = face_classifier.detectMultiScale(gray,1.3,5)
                cropped_face = img
                if faces is ():
                    return None
                for(x,y,w,h) in  faces:
                    cropped_face = img[y:y+h,x:x+w]
                return cropped_face

            count = 0
            cap = cv2.VideoCapture(0)
            temp = 1
            id = en_no
            img_id = 0
            while True:
                ret , frame = cap.read()
                if face_cropped(frame) is not None:
                    img_id += 1
                    face = cv2.resize(face_cropped(frame),(200,200))
                    face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
                    path = "faces/user."+id+"."+str(img_id)+".jpg"
                    cv2.imwrite(path,face)
#                     cv2.putText(face,"Camera Shut off Automatically at count 80...",(30,30),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
                    cv2.putText(face,str(img_id),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
                    cv2.imshow("Cropped Face",face)
                    if cv2.waitKey(1) == 13 or count == 80:
                        break
                    count =  count + 1  
            cap.release()
            cv2.destroyAllWindows()
            
            mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition")
            mycursor = mydb.cursor()
            query = "UPDATE `id_generator` SET `next_id` = `next_id` + 1 WHERE 1"
            mycursor.execute(query)
            mydb.commit()
            mycursor.execute("select * from id_generator")
            id_label['text'] = mycursor.fetchone()[0]
            
            train_model()
            details(en_no,name,ty_pe,department,email,phn,address,pincode)

<>:53: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:53: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-15-bcc73c18243f>:53: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In [16]:
def download_attendence():
    mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition")
    today = datetime.date.today()
    isExists = os.path.exists("Year "+str(today.year))
    if isExists:
        os.chdir("Year "+str(today.year))
    else :
        os.mkdir("Year "+str(today.year))
        os.chdir("Year "+str(today.year))
    df = pd.read_sql(sql="SELECT * FROM `attendence` ",con = mydb)
    df.to_excel(str(today.strftime("%B"))+".xlsx")
    os.chdir("..")
    os.getcwd()
    messagebox.showinfo('Message','Attendence Taken Successfully...')

In [17]:
def new_user():
    
    F9.place_forget()
    b1_df.place_forget()
    label1_img.place_forget()
    
    e5.place_forget()
    b3_gd.place_forget()
    b4_da.place_forget()
    label3_img.place_forget()
    b5_da.place_forget()

    
    frame3.place_forget()
    frame4.place_forget()
    datef.pack_forget()

    clock.pack_forget()
    
    j1.place_forget()
    s1.place_forget()
    
    d1.place_forget()
    
    j2.place_forget()
    s2.place_forget()
    
    d2.place_forget()
    label2_img.place_forget()
    
    message3 = tk.Label(window, text="Employee Details" ,fg="white",bg="#262523" ,width=36 ,height=1,font=('times', 17, ' bold '))
    message3.place(x=0, y=0)
    
    t8.place(x=210,y=530)
    
    t3.place(x=60, y = 60) 
    id_label.place(x=230, y = 70)
    
    t2.place(x=60, y = 120) 
    a2.place(x=240,y=130)
    
    t1.place(x=60, y = 180)
    a1.place(x=240, y = 190)
    
    t4.place(x=60, y = 240) 
    a4.place(x=240, y = 250)
    
    t5.place(x=60, y = 300) 
    a5.place(x=240, y = 310)
    
    t6.place(x=60, y = 360) 
    a6.place(x=240, y = 370)
    
    t7.place(x=60, y = 420) 
    a7.place(x=240, y = 430)
    
    t9.place(x=60, y = 480) 
    a9.place(x=240, y = 490)

def home():
    t1.place_forget()
    a1.place_forget()
    
    t2.place_forget()
    a2.place_forget()
    
    t3.place_forget()
    id_label.place_forget()
    
    t4.place_forget()
    a4.place_forget()
    
    t5.place_forget()
    a5.place_forget()
    
    t6.place_forget()
    a6.place_forget()
    
    t7.place_forget()
    a7.place_forget()
    
    t9.place_forget()
    a9.place_forget()
    
    t8.place_forget()
    
    e5.place_forget()
    b3_gd.place_forget()
    b4_da.place_forget()
    label3_img.place_forget()
    b5_da.place_forget()
    
    j1.place_forget()
    s1.place_forget()
    
    d1.place_forget()
    j2.place_forget()
    s2.place_forget()
    
    d2.place_forget()
    label2_img.place_forget()
    
    label1_img.place(x=130, y=90)
    b1_df.place(x = 200, y = 330)
    
    
    F9.place(x=0.3,y=450,width=520,height=200)

def admin():
    F9.place_forget()
    b1_df.place_forget()
    label1_img.place_forget()
    
    t1.place_forget()
    a1.place_forget()
    
    t2.place_forget()
    a2.place_forget()
    
    t3.place_forget()
    id_label.place_forget()
    
    t4.place_forget()
    a4.place_forget()
    
    t5.place_forget()
    a5.place_forget()
    
    t6.place_forget()
    a6.place_forget()
    
    t7.place_forget()
    a7.place_forget()
    
    t9.place_forget()
    a9.place_forget()
    
    t8.place_forget()
    
    j1.place_forget()
    s1.place_forget()
    
    d1.place_forget()
    
    j2.place_forget()
    s2.place_forget()
    
    d2.place_forget()
    
    e5.place(x=170,y=480)
    b3_gd.place(x=188,y=360) 
    b4_da.place(x=30,y=420)
    
    b5_da.place(x=300,y=420)
    label3_img.place(x=150, y=90)
            
def login_page():
        if(id1.get()=="1231"):
            if(pass1.get()=="milan"):
                admin()
                mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition")
                mycursor = mydb.cursor()
                sql = "update attendence set `%s` = 'present' where enrollment_no = %s"
                val = (str(today),str(id1.get()),)
                mycursor.execute(sql,val)
                mydb.commit()
            else:
                messagebox.showerror("Error","invalid password")
        else:
            messagebox.showerror("Error","invalid User id")
            
def exit_app():
    op1=messagebox.askyesno("Exit","Do you want to Exit")
    if op1>0:
        window.destroy()
    else:
        return
    
def contact():
    mess._show(title='Contact us', message="Please contact us on : \n\n'milanraghvani75@gmail.com' \n\t      or \n'sambhavanisantosh@gmail.com' ")
    

def manual_attendance():
    
    F9.place_forget()
    b1_df.place_forget()
    label1_img.place_forget()
    
    t1.place_forget()
    a1.place_forget()
    
    t2.place_forget()
    a2.place_forget()
    
    t3.place_forget()
    id_label.place_forget()
    
    t4.place_forget()
    a4.place_forget()
    
    t5.place_forget()
    a5.place_forget()
    
    t6.place_forget()
    a6.place_forget()
    
    t7.place_forget()
    a7.place_forget()
    
    t9.place_forget()
    a9.place_forget()
    
    t8.place_forget()
    
    e5.place_forget()
    b3_gd.place_forget()
    b4_da.place_forget()
    label3_img.place_forget()
    b5_da.place_forget()
      
    j1.place(x=50, y = 350) 
    s1.place(x=260, y = 360)
    
    d1.place(x=200,y=450)
    label2_img.place(x=130, y=120)
    
    
def otp():
    
    server = smtplib.SMTP_SSL("smtp.gmail.com", 465)    
    user = "milanraghvani75@gmail.com"
    password = "opspfjtkvtohcjeh"
    if(s1.get() == ""):
        messagebox.showinfo('Message','Please Enter the Employee Id')
    else:
        dest = s1.get()
        print(dest)
        
        mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition")
        mycursor = mydb.cursor()
        sql = "select count(*) from users where enrollment_no = %s"
        val = (str(dest),)
        mycursor.execute(sql,val)
        count = mycursor.fetchone()
        print(count)
        if count[0] == 1:
            messagebox.showinfo('Message','OTP has been sent to your register email id')
            d1.place_forget()
            j2.place(x=50, y = 410) 
            s2.place(x=260, y = 420)
            d2.place(x=200,y=500)
            
            mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition")
            mycursor = mydb.cursor()
            sql = "select email from users where enrollment_no = %s"
            val1 = (str(dest),)
            mycursor.execute(sql,val1)
            dest1 = mycursor.fetchone()
            print(dest1)
            mydb.commit()
            server.login(user, password)

            data = "0123456789"

            leng = len(data)
            global otp1
            otp1 = ""

            for i in range(6):
                otp1 += data[math.floor(random.random()*leng)]

            print("your otp is",otp1)

            server.sendmail("milanraghvani75@gmail.com",
                            dest1,
                            otp1+" is your verification code for Leave management manually attendance.")

            server.quit()
        else :
            messagebox.showinfo('Message','User Id Does not Exist')
        
        
def submit(otp1):
    print(otp1,s2.get())
    if(s2.get() == otp1):
        messagebox.showinfo('Message','Your Request Sent Successfully')
        mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition")
        mycursor = mydb.cursor()
        sql = "select name from users where enrollment_no = %s"
        val1 = (str(s1.get()),)
        mycursor.execute(sql,val1)
        db_name = mycursor.fetchone()
        mydb.commit()
        mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition")
        mycursor = mydb.cursor()
        today = datetime.date.today() 
        sql = "INSERT INTO `manualreports` (`id`,`name`,`date`) VALUES (%s,%s,%s)"
        value = (s1.get(),db_name[0],today,)
        mycursor.execute(sql,value)
        mydb.commit()
    elif(s2.get() == ""):
        messagebox.showinfo('Message','Please Provide OTP')
    else:
        messagebox.showinfo('Message','Entered OTP Is Wrong')
        


In [18]:
#---------------Manual attendance Gui-----------------------


def gui():
    r = tk.Tk()
    r.title("Manual attendence practice")
    r.geometry("800x500+100+100")

    mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition") 
    mycursor = mydb.cursor() 
    mycursor.execute("select * from manualreports")

    tree = ttk.Treeview(r)
    tree['show'] = 'headings'

    s = ttk.Style(r)
    s.theme_use("clam")
    s.configure(".", font=('Helvetica',11))
    s.configure("Treeview.Heading",font=('Helvetica', 13))

    tree["columns"] = ("id", "name", "date")

    tree.column("id", width=130, minwidth=130, anchor=tk.CENTER)
    tree.column("name", width=150, minwidth=150, anchor=tk.CENTER)
    tree.column("date", width=130, minwidth=130, anchor=tk.CENTER)

    tree.heading("id", text="Employee ID", anchor=tk.CENTER)
    tree.heading("name", text="Employee Name", anchor=tk.CENTER)
    tree.heading("date", text="Date", anchor=tk.CENTER)

    i = 0
    for ro in mycursor:
        tree.insert('',i,text = "", values=(ro[0],ro[1],ro[2]))
        i = i + 1

    hsb = ttk.Scrollbar(r,orient="horizontal")
    hsb.configure(command=tree.xview)
    tree.configure(xscrollcommand=hsb.set)
    hsb.pack(fill=X, side=BOTTOM)

    tree.pack()

    def selected_record():
        selected_item = tree.focus()
        print(selected_item[0])
        values =tree.item(selected_item,'values')
        print(values)

    select_button = tk.Button(r,text="select_record",command=selected_record)

    def accept():
        values =tree.item(tree.focus(),'values')
        id = values[0]
        date = values[2]
        print(id,date)
        mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition") 
        mycursor = mydb.cursor() 
        mycursor.execute("update attendence set `%s` = 'present' where enrollment_no = %s",(date,id))
        mycursor.execute("delete from manualreports where id = %s",(id,))
        mydb.commit()
        tree.delete(tree.focus())
    #         download_attendence()

        os.chdir("Year "+str(today.year))
        df = pd.read_excel(str(today.strftime("%B"))+".xlsx", engine='openpyxl')
        df.drop("Unnamed: 0",inplace = True,axis = 'columns')
        print(df)
        df.set_index('enrollment_no',inplace=True)
        df.at[int(id),"'"+date+"'"] = 'present'
        print(df)
        df.to_excel(str(today.strftime("%B"))+".xlsx", engine='openpyxl')
        os.chdir("..")  

    def reject():
        values =tree.item(tree.focus(),'values')
        id = values[0]
        mycursor.execute("delete from manualreports where id = %s",(id,))
        mydb.commit()
        tree.delete(tree.focus())

    accept = tk.Button(r,text="accept",width=9,font=("Helvetica",15,"bold"),bg="#31618D",fg="white",command=accept)
    accept.place(x=250,y=300)
    reject = tk.Button(r,text="reject",width=9,font=("Helvetica",15,"bold"),bg="#31618D",fg="white",command=reject)
    reject.place(x=430,y=300)

    r.mainloop()

In [19]:
#---------------Search Gui-----------------------

def gui2():
    months=['January','February','March','April','May','June','July','August','September','October','November','December']
    r_1 = tk.Tk()
    r_1.title("Search attendence practice")
    r_1.geometry("800x500+100+100")
    today = datetime.date.today()

    def selectItem(a,id):
        df = pd.read_excel("Year 2021/"+str(month_selected.get())+".xlsx", engine='openpyxl')
        df.drop("Unnamed: 0",inplace = True,axis = 'columns')
        df.drop("enrollment_no",inplace = True,axis = 'columns')
        my_tree['column'] = list(df.columns)
        my_tree['show'] = 'headings'

        for column in my_tree['column']:
            my_tree.column(column,  anchor=tk.CENTER)
            my_tree.heading(column, text = column, anchor=tk.CENTER)

        df_rows = a.to_numpy().tolist()

        print(df_rows)
        my_tree.insert("","end",values=df_rows)

        my_tree.pack()
        curItem = r_1.focus()
        print(curItem, a)

    def clear_tree():
        my_tree.delete(*my_tree.get_children())

    def search():
        os.chdir("Year "+str(today.year))
        df = pd.read_excel(str(month_selected.get())+".xlsx", engine='openpyxl')
        df.drop("Unnamed: 0",inplace = True,axis = 'columns')
        os.chdir("..")
        clear_tree()
        if idEntry.get() == "":
            my_tree['column'] = list(df.columns)
            my_tree['show'] = 'headings'

            for column in my_tree['column']:
                my_tree.column(column,  anchor=tk.CENTER)
                my_tree.heading(column, text = column, anchor=tk.CENTER)

            df_rows = df.to_numpy().tolist()

            for row in df_rows:
                my_tree.insert("","end",values=row)

            my_tree.pack()

        else :
            id = int(idEntry.get())
            df.set_index('enrollment_no',inplace=True)
            df = df.loc[id]
            selectItem(df,id)

    x = today.month - 1

    idLable = Label(r_1,text="Id",font=("Helvetica",20,"bold"),bg="#F3E3AE",fg="#383636")
    idLable.pack()
    idEntry  = Entry(r_1,width=30,bd=3)
    idEntry.pack()

    month_selected = StringVar()
    month_selected.set(months[x])

    month_dropDown = OptionMenu(r_1,month_selected,*months).pack()

    Search_button = tk.Button(r_1,text="Search" , command = search).pack()

    my_frame = Frame(r_1)
    my_frame.pack(pady=20)

    my_tree = ttk.Treeview(my_frame)

    hsb = ttk.Scrollbar(r_1,orient="horizontal")
    hsb.configure(command=my_tree.xview)
    my_tree.configure(xscrollcommand=hsb.set)
    hsb.pack(fill=X, side=BOTTOM)

    r_1.mainloop()

In [20]:
global window
window = tk.Tk()
window.title("Leave Management")
window.configure(bg="white")

today = datetime.date.today()

if (today.day == 1):
    mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition")
    mycursor = mydb.cursor()
    mycursor.execute("drop table attendence")
    mycursor.execute("create table if not exists attendence(enrollment_no varchar(14) primary key, name varchar(50), `%s` varchar(32) default 'absent')",(str(today),))
    mycursor.execute("insert into attendence(enrollment_no, name) select enrollment_no,name from users")
    mydb.commit()
try:
    today = datetime.date.today()
    mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition") 
    mycursor = mydb.cursor() 
    addColumn = 'ALTER TABLE attendence ADD COLUMN `%s` varchar(11) default "absent"' 
    val = (str(today),) 
    mycursor.execute(addColumn,val)
except :
    print("column already exist")
        
mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition") 
mycursor = mydb.cursor() 
query = 'select * from id_generator' 
mycursor.execute(query)
new_id = mycursor.fetchone()

mymenu = Menu(window, tearoff=0)
mymenu.add_command(label="Home", command=home)
mymenu.add_command(label="Administrator", command = lambda : detect_face("admin"))
mymenu.add_command(label="Manually Attendance",command=manual_attendance)
mymenu.add_command(label="Help", command=contact)
mymenu.add_command(label="Exit", command=exit_app)

window.config(menu=mymenu)

message3 = tk.Label(window, text="Face Recognition Based Attendance System" ,fg="white",bg="#262523" ,width=37 ,height=1,font=('times', 17, ' bold '))
message3.place(x=0, y=0)

frame3 = tk.Frame(window, bg="#c4c6ce")
frame3.place(relx=0.48, rely=0.09, relwidth=0.42, relheight=0.07)

frame4 = tk.Frame(window, bg="#c4c6ce")
frame4.place(relx=0.27, rely=0.09, relwidth=0.32, relheight=0.07)

datef = tk.Label(frame4, text = day+"-"+mont[month]+"-"+year+"     |  ", fg="blue",bg="white" ,width=50 ,height=1,font=('times', 15, ' bold '))
datef.pack(fill='both',expand=1)

clock = tk.Label(frame3,fg="blue",bg="white" ,width=50 ,height=1,font=('times', 15, ' bold '))
clock.pack(fill='both',expand=1)
tick()

# admin page...

e5 = tk.Button(window,text="Search Employee",font=("Helvetica",15,"bold"),command=gui2,bg="#31618D",fg="white") 
b3_gd = tk.Button(window,text="New Register",font=("Helvetica",15,"bold"),command=new_user,bg="#31618D",fg="white") 
b4_da = tk.Button(window,text="Download Attendence",font=("Helvetica",15,"bold"),command=download_attendence,bg="#31618D",fg="white") 

b5_da = tk.Button(window,text="Manual Attendance",font=("Helvetica",15,"bold"),command=gui,bg="#31618D",fg="white") 

image3 = Image.open("admin.png")
test3 = ImageTk.PhotoImage(image3)
label3_img = tk.Label(image=test3)
label3_img.image = test3


# home page...
image1 = Image.open("images.jpg")
test = ImageTk.PhotoImage(image1)
label1_img = tk.Label(image=test)
label1_img.image = test
label1_img.place(x=130, y=90)

b1_df = tk.Button(window,text="Detect Face",font=("Helvetica",15,"bold"),bg="green",fg="white",command=lambda: detect_face("emp"))
b1_df.place(x = 200, y = 330)

#---------------Admin area-----------------------

F9=LabelFrame(window,bd=10,relief=GROOVE,text="Admin area ",font=("times new roman",15,"bold"),fg="black",bg="gold")
F9.place(x=0.3,y=450,width=520,height=200)

Label(F9,text="Login :  | ",font=("times new roman",15,"bold"),fg="black",bg="gold").grid(row=0,column=0)
Label(F9,text="ID ",font=("times new roman",15,"bold"),fg="black",bg="gold").place(x=100,y=5)
        
id1=tk.Entry(F9,width=20,font="arial 10 bold",bd=7,relief=SUNKEN)
id1.place(x=180,y=3)

Label(F9,text="  Password",font=("times new roman",15,"bold"),fg="black",bg="gold").place(x=70,y=50)

pass1=tk.Entry(F9,width=20,font="arial 10 bold",bd=7,relief=SUNKEN,show='X')
pass1.place(x=180,y=50)

a_login=Button(F9,text="Login",bg="cyan",bd=5,fg="black",width=12,font="arial 12 bold",command=login_page).place(x=70,y=100)

#---------------Manual Attendance area-----------------------

j1 = tk.Label(window,text="Employee Id",font=("Helvetica",20,"bold"),bg="#F3E3AE",fg="#383636")
s1 = tk.Entry(window,width=30,bd=3)

j2 = tk.Label(window,text="OTP",font=("Helvetica",20,"bold"),bg="#F3E3AE",fg="#383636")
s2 = tk.Entry(window,width=30,bd=3)

d1 = tk.Button(window,text="Send OTP",font=("Helvetica",15,"bold"),command=otp,bg="#31618D",fg="white")
d2 = tk.Button(window,text="Submit",font=("Helvetica",15,"bold"),command=lambda: submit(otp1),bg="#31618D",fg="white")

image2 = Image.open("images.jpg")
test1 = ImageTk.PhotoImage(image2)
label2_img = tk.Label(image=test1)
label2_img.image = test1

#---------------New Register area-----------------------

t3 = tk.Label(window,text="Id",font=("Helvetica",20,"bold"),bg="#F3E3AE",fg="#383636") 
id_label = tk.Label(window,text=str(new_id[0]) ,font=("Helvetica",20,"bold"),bg="#FFF",fg="#383636")
# a3 = tk.Entry(window,width=30,bd=3) 

t2 = tk.Label(window,text="Name",font=("Helvetica",20,"bold"),bg="#F3E3AE",fg="#383636") 
a2 = tk.Entry(window,width=30,bd=3)  

t1 = tk.Label(window,text="Type",font=("Helvetica",20,"bold"),bg="#F3E3AE",fg="#383636")  
a1 = tk.Entry(window,width=30,bd=3)  

t4 = tk.Label(window,text="Department",font=("Helvetica",20,"bold"),bg="#F3E3AE",fg="#383636") 
a4 = tk.Entry(window,width=30,bd=3)

t5 = tk.Label(window,text="Email Id",font=("Helvetica",20,"bold"),bg="#F3E3AE",fg="#383636") 
a5 = tk.Entry(window,width=30,bd=3)

t6 = tk.Label(window,text="Phone No.",font=("Helvetica",20,"bold"),bg="#F3E3AE",fg="#383636") 
a6 = tk.Entry(window,width=30,bd=3)

t7 = tk.Label(window,text="Address",font=("Helvetica",20,"bold"),bg="#F3E3AE",fg="#383636") 
a7 = tk.Entry(window,width=30,bd=3)

t9 = tk.Label(window,text="Pincode",font=("Helvetica",20,"bold"),bg="#F3E3AE",fg="#383636") 
a9 = tk.Entry(window,width=30,bd=3)

t8 = tk.Button(window,text="Submit",font=("Helvetica",15,"bold"),command=generate_dataset,bg="#31618D",fg="white")


window.geometry("520x650+200+30")
window.resizable(True, True)
window.mainloop()



column already exist


In [21]:
# os.chdir("C:\\Users\\Sam\\Documents\\Artificial Intelligernce\\")
# os.getcwd()

In [22]:
# import time
# # print(time.strftime("%H"))
# today = datetime.date.today() 
# print(today)
# add = 'first_' 
# m = add+count 

In [23]:
# mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition") 
# mycursor = mydb.cursor() 
# query = 'select * from id_generator' 
# mycursor.execute(query)
# next_id = mycursor.fetchone()
# print(next_id[0])

In [24]:
# mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition")
# mycursor = mydb.cursor()
# query = "UPDATE `id_generator` SET `next_id` = `next_id` + 1 WHERE 1"
# mycursor.execute(query)
# mydb.commit()
# mycursor.execute("select * from id_generator")
# print(mycursor.fetchone())

In [25]:
# today = datetime.date.today()
# mydb = mysql.connector.connect(host="localhost",user="root",passwd="",database="facerecognition")
# mycursor = mydb.cursor()
# mycursor.execute("create table if not exists attendence_1(enrollment_no varchar(14) primary key, name varchar(50), `%s` varchar(32) default 'absent')",(str(today),))
# mycursor.execute("insert into attendence_1(enrollment_no, name) select enrollment_no,name from users")
# mydb.commit()